<a href="https://colab.research.google.com/github/AshOne91/PLAYDATA-COLAB/blob/main/LLM%EC%9D%98_%EC%9D%91%EC%9A%A9(LLM_%EC%A0%81%EC%9A%A9%ED%95%98%EA%B8%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rag Langchain을 이용해서 한글문서를 기반으로 대화 맥락을 유지하면서 문서를 참조하는 챗봇
```
RAG : Retrieval Argumented Generation
외부문서를 검색(Retrieval) 해서 LLM의 답변생성(Generation)을 강화하는 기술
  문서검색 : 질문과 관련된 문서를 벡터 데이터베이스에서 검색
  문서임베딩 : 텍스트를 벡터화(의미적 유사성)
  답변생성 : 검색된 문서를 LLM전달해서 맥락에 맞는 답변 생성
  장점 :
  한글고려사항 : 다국어 임베딩등 다양한 임베딩 기법을 사용  
LangChain : 오픈소스 프레임웍- LLM어플리케이션개발을 단순화
  Prompt Templates : 동적 프롬프트 생성
  Memory : 대화맥락 유지
  Chains : 작업흐름관리
  Retrival : 외부문서 검색
  한글지원 잘됨, openai api와 호환
  문서로드 : TextLoader-> Document객체 변환- 이후 문서를 작은단위로 분할(청크)
```

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = '...'

In [ ]:
!pip install langchain -q

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
# 문서 로드
loader = TextLoader('/content/korea_culture.txt')
documents = loader.load()
documents

[Document(metadata={'source': '/content/korea_culture.txt'}, page_content='대한민국은 동아시아에 위치한 나라로, 공식 명칭은 대한민국입니다.\n한국의 수도는 서울이며, 서울은 약 970만 명의 인구를 가진 대도시입니다.\n한국의 공식 언어는 한글로, 세종대왕이 1443년에 창제했습니다.\n한국의 전통 음식으로는 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.\n한국은 K-팝과 드라마로 전 세계적으로 문화적 영향력을 확대하고 있습니다.\n한국의 주요 명절로는 설날과 추석이 있으며, 가족들이 모여 전통 음식을 나눕니다.\n한국의 전통 의상인 한복은 색상과 디자인이 화려하며, 명절이나 결혼식에서 자주 입습니다.')]

In [ ]:
# 문서 분할
text_splitter =  RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50,
                                                separators=['\n\n', '\n', '.', '!', '?', ',', ' '],
                                                keep_separator=True)
texts = text_splitter.split_documents(documents)
texts

[Document(metadata={'source': '/content/korea_culture.txt'}, page_content='대한민국은 동아시아에 위치한 나라로, 공식 명칭은 대한민국입니다.\n한국의 수도는 서울이며, 서울은 약 970만 명의 인구를 가진 대도시입니다.'),
 Document(metadata={'source': '/content/korea_culture.txt'}, page_content='한국의 수도는 서울이며, 서울은 약 970만 명의 인구를 가진 대도시입니다.\n한국의 공식 언어는 한글로, 세종대왕이 1443년에 창제했습니다.'),
 Document(metadata={'source': '/content/korea_culture.txt'}, page_content='한국의 공식 언어는 한글로, 세종대왕이 1443년에 창제했습니다.\n한국의 전통 음식으로는 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.'),
 Document(metadata={'source': '/content/korea_culture.txt'}, page_content='한국의 전통 음식으로는 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.\n한국은 K-팝과 드라마로 전 세계적으로 문화적 영향력을 확대하고 있습니다.'),
 Document(metadata={'source': '/content/korea_culture.txt'}, page_content='한국은 K-팝과 드라마로 전 세계적으로 문화적 영향력을 확대하고 있습니다.\n한국의 주요 명절로는 설날과 추석이 있으며, 가족들이 모여 전통 음식을 나눕니다.'),
 Document(metadata={'source': '/content/korea_culture.txt'}, page_content='한국의 주요 명절로는 설날과 추석이 있으며, 가족들이 모여 전통 음식을 나눕니다.\n한국의 전통 의상인 한복은 색상과 디자인이 화려하며, 명절이나 결혼식에서 자주 입습니다.')]

In [ ]:
# 문서의 내용 및 길이 확인
with open('/content/korea_culture.txt', encoding='utf-8') as f:
  text = f.read()
  print(f'문서길이 : {len(text)}')
  temp = text.count('\n')
  print(f"줄바꿈 수 : {temp}")
  print(f"구분점 수 : {text.count('.')}")

문서길이 : 294
줄바꿈 수 : 6
구분점 수 : 7


In [ ]:
def load_and_seperare_document(texts = 'korea_culture.txt',chunk_size=100,chunk_overlap=10):
  # 문서로드
  loader = TextLoader(texts)
  documents = loader.load()
  # 문서내용 및 길이 확인
  with open(texts, encoding='utf-8') as f:
    text = f.read()
    print(f'문서길이 : {len(text)}')
    temp = text.count('\n')
    print(f"줄바꿈 수 : {temp}")
    print(f"구분점 수 : {text.count('.')}")
  # 문서 분할
  # 분할된 청크를 반환
  text_splitter =  RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap,
                                                separators=['\n\n', '\n', '.', '!', '?', ',', ' '],
                                                keep_separator=True)
  texts = text_splitter.split_documents(documents)
  print(f'분할된 청크수 : {len(texts)}')
  for i,chunk in enumerate(texts):
    print(f'청크{i+1} : {chunk.page_content}')
  return texts

In [ ]:
chunks = load_and_seperare_document()

문서길이 : 294
줄바꿈 수 : 6
구분점 수 : 7
분할된 청크수 : 4
청크1 : 대한민국은 동아시아에 위치한 나라로, 공식 명칭은 대한민국입니다.
한국의 수도는 서울이며, 서울은 약 970만 명의 인구를 가진 대도시입니다.
청크2 : 한국의 공식 언어는 한글로, 세종대왕이 1443년에 창제했습니다.
한국의 전통 음식으로는 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.
청크3 : 한국은 K-팝과 드라마로 전 세계적으로 문화적 영향력을 확대하고 있습니다.
한국의 주요 명절로는 설날과 추석이 있으며, 가족들이 모여 전통 음식을 나눕니다.
청크4 : 한국의 전통 의상인 한복은 색상과 디자인이 화려하며, 명절이나 결혼식에서 자주 입습니다.


RAG 구현
```
  벡터데이터베이스 Chroma 가볍고 빠름 -> Langchain과잘 어울림
  임베딩 : OepnAiEmbedding : 다국어 지원
  검색 ; 질문도 벡터화해서 데이터베이스에 저장된 벡터와 유사도 검색

  RetrivalQA체인:
    질문->벡터데이터베이스 에서 문서 검색-> 검색된 문서를 LLM에 전달 ->답변
```


In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 8.6 MB/s eta 0:

In [ ]:
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
import os
os.environ['OPENAI_API_KEY'] = '...'

In [ ]:
# LLM 초기화
llm = OpenAI(openai_api_key=os.environ['OPENAI_API_KEY'])
# 벡터 데이터베이스 생성
embeddings = OpenAIEmbeddings()
texts = load_and_seperare_document()  # 청크분할
vectorstore =  Chroma.from_documents(texts, embeddings, persist_directory='./db')
# vectorstore.persist()  # 벡터 데이터 베이스 생성 완료  ince Chroma 0.4.x 이후에서는 자동으로 DB가 저장된다

문서길이 : 294
줄바꿈 수 : 6
구분점 수 : 7
분할된 청크수 : 4
청크1 : 대한민국은 동아시아에 위치한 나라로, 공식 명칭은 대한민국입니다.
한국의 수도는 서울이며, 서울은 약 970만 명의 인구를 가진 대도시입니다.
청크2 : 한국의 공식 언어는 한글로, 세종대왕이 1443년에 창제했습니다.
한국의 전통 음식으로는 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.
청크3 : 한국은 K-팝과 드라마로 전 세계적으로 문화적 영향력을 확대하고 있습니다.
한국의 주요 명절로는 설날과 추석이 있으며, 가족들이 모여 전통 음식을 나눕니다.
청크4 : 한국의 전통 의상인 한복은 색상과 디자인이 화려하며, 명절이나 결혼식에서 자주 입습니다.


In [ ]:
# RetrivalQA 체인 생성
qa_chain =  RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = 'stuff', # 작은 문서는 분할하지않고 한번에 전달
    retriever = vectorstore.as_retriever(search_kwargs = {'k':2}),  # 상위 2개 문서 검색
    return_source_documents = True,
)

In [ ]:
# 질문 생성
query = "미국의 공식 언어는 무엇입니까?"
result = qa_chain({'query':query})
# 결과 출력
print(f'질문 : {query}')
print(f'답변 : {result["result"]}')
print(f'참조 문서 : {[doc.page_content for doc in result["source_documents"]]}')

질문 : 미국의 공식 언어는 무엇입니까?
답변 :  미국의 공식 언어는 영어입니다.
참조 문서 : ['한국의 공식 언어는 한글로, 세종대왕이 1443년에 창제했습니다.\n한국의 전통 음식으로는 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.', '한국의 공식 언어는 한글로, 세종대왕이 1443년에 창제했습니다.\n한국의 전통 음식으로는 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.']


In [ ]:
# 단순 llm을 사용
llm.invoke(query)

'\n\n미국의 공식 언어는 없지만, 영어가 사실상의 공식 언어로 사용되고 있습니다.'

1. 문서선택(text, pdf ,doc 등 다양한 문서 검색 기능을 지원)
2. 문서를 기반으로 벡터데이터 베이스 생성 - RAG
3. RAG를 기반으로 랭체인 구축(RAG를 통해 검색을 해서 해당 데이터를 LLM 전달 랭체인)

In [ ]:
# 회사규정
'''
우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.
근태관리 기준은 8출근 5퇴근을 기준으로 한다
금요일은 전사가 오전 근무만 한다
재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다
'''

'\n우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.\n근태관리 기준은 8출근 5퇴근을 기준으로 한다\n금요일은 전사가 오전 근무만 한다\n재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다\n'

In [ ]:
llm.invoke("우리회사의 재택근무는 어떻게 되나요?")

'\n저희 회사에서는 재택근무를 위한 시스템과 규정을 마련하고 있습니다. 재택근무는 업무의 특성상 가능한 경우에만 허용하고 있으며, 상사의 승인을 받아야 합니다.\n\n또한 재택근무를 할 때에도 근무시간, 출퇴근 기록 등 업무 관련 모든 사항은 업무시간 외에도 자율적으로 관리해야 합니다. 이를 위해 저희 회사에서는 재택근무 규정을 제공하고, 업무시간 외에도 업무 관련 사항을 관리할 수 있는 시스템을 제공하고 있습니다.\n\n또한 재택근무를 할 때에도 업무의 목적과 내용'

In [ ]:
# 회사규정
company_rules = '''
우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.
근태관리 기준은 8출근 5퇴근을 기준으로 한다
금요일은 전사가 오전 근무만 한다
재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다
'''
with open('company_rules.txt', 'w') as f:
  f.write(company_rules)

# LLM 초기화
llm = OpenAI(openai_api_key=os.environ['OPENAI_API_KEY'])
# 벡터 데이터베이스 생성
embeddings = OpenAIEmbeddings()
texts = load_and_seperare_document('company_rules.txt',chunk_size=50)  # 청크분할
vectorstore =  Chroma.from_documents(texts, embeddings, persist_directory='./db3')
# RetrivalQA 체인 생성
qa_chain =  RetrievalQA.from_chain_type(
    llm = llm,
    # chain_type = 'stuff', # 작은 문서는 분할하지않고 한번에 전달
    retriever = vectorstore.as_retriever(search_kwargs = {'k':5}),  # 상위 2개 문서 검색
    return_source_documents = True,
)
# 질문 생성
query = "우리회사의 재택근무는 어떻게 되나요?"
result = qa_chain({'query':query})
# 결과 출력
print(f'질문 : {query}')
print(f'답변 : {result["result"]}')
print(f'참조 문서 : {[doc.page_content for doc in result["source_documents"]]}')

문서길이 : 122
줄바꿈 수 : 5
구분점 수 : 1
분할된 청크수 : 3
청크1 : 우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.
청크2 : 근태관리 기준은 8출근 5퇴근을 기준으로 한다
금요일은 전사가 오전 근무만 한다
청크3 : 재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다
질문 : 우리회사의 재택근무는 어떻게 되나요?
답변 : 

정답: 재택근무는 원하는 경우 승인 없이 자유롭게 실시할 수 있습니다.
참조 문서 : ['우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.', '우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.', '재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다', '재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다', '근태관리 기준은 8출근 5퇴근을 기준으로 한다\n금요일은 전사가 오전 근무만 한다']


In [ ]:
# 청크사이즈, 참조문서의 개수 --> 파이퍼 파라메터
!pip install -U langchain-community -q
!pip install chromadb -q
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
import os
import shutil
os.environ['OPENAI_API_KEY'] = '...'

In [ ]:
def load_and_seperare_document(texts = 'korea_culture.txt',chunk_size=100,chunk_overlap=10):
  # 문서로드
  loader = TextLoader(texts)
  documents = loader.load()
  # 문서내용 및 길이 확인
  with open(texts, encoding='utf-8') as f:
    text = f.read()
    print(f'문서길이 : {len(text)}')
    temp = text.count('\n')
    print(f"줄바꿈 수 : {temp}")
    print(f"구분점 수 : {text.count('.')}")
  # 문서 분할
  # 분할된 청크를 반환
  text_splitter =  RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap,
                                                separators=['\n\n', '\n', '.', '!', '?', ',', ' '],
                                                keep_separator=True)
  texts = text_splitter.split_documents(documents)
  print(f'분할된 청크수 : {len(texts)}')
  for i,chunk in enumerate(texts):
    print(f'청크{i+1} : {chunk.page_content}')
  return texts

In [ ]:
if os.path.exists('./db001'):
  shutil.rmtree('./db001')

In [ ]:
# 회사규정
company_rules = '''
우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.
근태관리 기준은 8출근 5퇴근을 기준으로 한다
금요일은 전사가 오전 근무만 한다
우리회사의 재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다
'''
with open('company_rules.txt', 'w') as f:
  f.write(company_rules)

# LLM 초기화
llm = OpenAI(openai_api_key=os.environ['OPENAI_API_KEY'])
# 벡터 데이터베이스 생성
embeddings = OpenAIEmbeddings()
texts = load_and_seperare_document('company_rules.txt',chunk_size=50,chunk_overlap=5)  # 청크분할
# 기존 벡터데이터베이스를 삭제하고 다시 저장
vectorstore =  Chroma.from_documents(texts, embeddings, persist_directory='./db2')
# RetrivalQA 체인 생성
qa_chain =  RetrievalQA.from_chain_type(
    llm = llm,
    # chain_type = 'stuff', # 작은 문서는 분할하지않고 한번에 전달
    retriever = vectorstore.as_retriever(search_kwargs = {'k':3}),  # 상위 2개 문서 검색
    return_source_documents = True,
)
# 질문 생성
query = "근태기준?"
result = qa_chain.invoke({'query':query})
# 결과 출력
print(f'\n\n질문 : {query}')
print(f'답변 : {result["result"]}')
print(f'참조 문서 : {[doc.page_content for doc in result["source_documents"]]}')

문서길이 : 128
줄바꿈 수 : 5
구분점 수 : 1
분할된 청크수 : 3
청크1 : 우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.
청크2 : 근태관리 기준은 8출근 5퇴근을 기준으로 한다
금요일은 전사가 오전 근무만 한다
청크3 : 우리회사의 재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다


질문 : 근태기준?
답변 :  근태관리 기준은 8출근 5퇴근을 기준으로 한다.
참조 문서 : ['근태관리 기준은 8출근 5퇴근을 기준으로 한다\n금요일은 전사가 오전 근무만 한다', '근태관리 기준은 8출근 5퇴근을 기준으로 한다\n금요일은 전사가 오전 근무만 한다', '근태관리 기준은 8출근 5퇴근을 기준으로 한다\n금요일은 전사가 오전 근무만 한다']


LangChain 으로 다양한 문서 포멧 불러오기

In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 50.5 MB/s eta 0:00:00


In [ ]:
# pdf 파일
from langchain.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("/content/company_rules.pdf")
pages = loader.load()  # 페이지 단위로 문서를 분리
pages


[Document(metadata={'producer': 'Microsoft® Word Microsoft 365용', 'creator': 'Microsoft® Word Microsoft 365용', 'creationdate': '2025-05-12T11:32:39+09:00', 'source': '/content/company_rules.pdf', 'file_path': '/content/company_rules.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': '', 'author': '이규영', 'subject': '', 'keywords': '', 'moddate': '2025-05-12T11:32:39+09:00', 'trapped': '', 'modDate': "D:20250512113239+09'00'", 'creationDate': "D:20250512113239+09'00'", 'page': 0}, page_content='우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다. \n근태관리 기준은 8출근 5퇴근을 기준으로 한다 \n금요일은 전사가 오전 근무만 한다 \n우리회사의 재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다 \n구분 \n종류 \n타입 \n가격 \n사무용 \n개인용품 \nA타입 \n5000')]

In [ ]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.9 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=78cbad7dc90109daef1b94fcabce829b90d6da3d23cb4d9837770d334e4985a8
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
# 워드문서 로드
from langchain.document_loaders import UnstructuredWordDocumentLoader
loader = UnstructuredWordDocumentLoader("/content/company_rules.docx")
pages = loader.load()  # 페이지 단위로 문서를 분리
pages


[Document(metadata={'source': '/content/company_rules.docx'}, page_content='우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.\n\n근태관리 기준은 8출근 5퇴근을 기준으로 한다\n\n금요일은 전사가 오전 근무만 한다\n\n우리회사의 재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다\n\n구분 종류 타입 가격 사무용 개인용품 A타입 5000')]

In [ ]:
import pandas as pd
data = {
    'name' : ['Alice', 'Bob', 'Charlie'],
    'age' : [25, 30, 35],
    'city' : ['New York', 'San Francisco', 'Los Angeles']
}
df = pd.DataFrame(data)
df.to_csv('example.csv', index=False)
df

,name,age,city
0,Alice,25,New York
1,Bob,30,San Francisco
2,Charlie,35,Los Angeles


In [ ]:
# csv 로드
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader("example.csv")
pages = loader.load()
pages


[Document(metadata={'source': 'example.csv', 'row': 0}, page_content='name: Alice\nage: 25\ncity: New York'),
 Document(metadata={'source': 'example.csv', 'row': 1}, page_content='name: Bob\nage: 30\ncity: San Francisco'),
 Document(metadata={'source': 'example.csv', 'row': 2}, page_content='name: Charlie\nage: 35\ncity: Los Angeles')]

In [ ]:
# 웹페이지 로드
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://github.com/sknetworks20250226/AI")
pages = loader.load()
pages

[Document(metadata={'source': 'https://github.com/sknetworks20250226/AI', 'title': 'GitHub - sknetworks20250226/AI', 'description': 'Contribute to sknetworks20250226/AI development by creating an account on GitHub.', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGitHub - sknetworks20250226/AI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu\n\nToggle navigation\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n            Sign in\n          \n\n\n\n\n\n\n\n\n        Product\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\nGitHub Copilot\n        Write better code with AI\n      \n\n\n\n\n\n\n\nGitHub Advanced Security\n        Find and fix vulnerabilities\n      \n\n\n\n\n\n\n\nActions\n        Automate any workflow\n      \n\n\n\n\n\n\n\nCodespaces\n        Instant dev envir